In [11]:
from keras import layers, models
from skeras import plot_loss, plot_acc
import matplotlib.pyplot as plt

In [6]:

class AE(models.Model):
    def __init__(self,x_nodes=784, z_dim=36): #default
        x = layers.Input(shape=(x_nodes,))
        z = layers.Dense(z_dim,activation='relu')(x)
        y = layers.Dense(x_nodes,activation='sigmoid')(z)
        
        super().__init__(x,y)
        
        self.x = x
        self.z = z
        self.z_dim = z_dim
            
        self.compile(optimizer='adadelta', loss='binary_crossentropy', 
                     metrics=['accuracy'])
    def Encoder(self):
        return models.Model(self.x, self.z)

    def Decoder(self):
        z = layers.Input((self.z_dim,))
        y_layer = self.layers[-1]
        y = y_layer(z)
        return models.Model(z, y)  
    
from keras.datasets import mnist
import numpy as np
(X_train, _), (X_test, _) = mnist.load_data()

X_train = X_train.astype('float32') / 255.
X_test = X_test.astype('float32') / 255.
X_train = X_train.reshape((len(X_train), np.prod(X_train.shape[1:])))
X_test = X_test.reshape((len(X_test), np.prod(X_test.shape[1:])))
print(X_train.shape)
print(X_test.shape)

def show_ae(autoencoder):
    encoder = autoencoder.Encoder()
    decoder = autoencoder.Decoder()

    encoded_imgs = encoder.predict(X_test)
    decoded_imgs = decoder.predict(encoded_imgs)

    n = 10
    plt.figure(figsize=(20, 6))
    for i in range(n):

        ax = plt.subplot(3, n, i + 1)
        plt.imshow(X_test[i].reshape(28, 28))
        plt.gray()
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)

        ax = plt.subplot(3, n, i + 1 + n)
        plt.stem(encoded_imgs[i].reshape(-1))
        plt.gray()
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)

        ax = plt.subplot(3, n, i + 1 + n + n)
        plt.imshow(decoded_imgs[i].reshape(28, 28))
        plt.gray()
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)

    plt.show()



def main():
    x_nodes = 784
    z_dim = 36

    autoencoder = AE(x_nodes, z_dim)

    history = autoencoder.fit(X_train, X_train,
                              epochs=10,
                              batch_size=256,
                              shuffle=True,
                              validation_data=(X_test, X_test))

    plot_acc(history, '(a) 학습 경과에 따른 정확도 변화 추이')
    plt.show()
    plot_loss(history, '(b) 학습 경과에 따른 손실값 변화 추이')
    plt.show()

    show_ae(autoencoder)
    plt.show()
if __name__ == '__main__':
    main()

(60000, 784)
(10000, 784)
Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 8s 136us/step - loss: 0.3491 - acc: 0.7591 - val_loss: 0.2690 - val_acc: 0.7950
Epoch 2/10
48128/60000 [=======================>......] - ETA: 1s - loss: 0.2622 - acc: 0.7969

KeyboardInterrupt: 